In [10]:
# PuLP is already installed, so let's jump in
import pandas as pd
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus

# Load dataset
url = "https://raw.githubusercontent.com/SAI-ADITY-A/Supply-chain-analysis/main/supply_chain_data.csv"
df = pd.read_csv(url)

# Preview data
print("Dataset Preview:")
print(df[['Product type', 'SKU', 'Shipping costs', 'Supplier name', 'Location']].head())

# Let's say we want to minimize shipping costs for selected SKUs
selected_skus = df['SKU'].unique()[:5]  # Just a few for demo
routes = [(sku, supplier) for sku in selected_skus for supplier in df['Supplier name'].unique()]
x = LpVariable.dicts("Route", routes, lowBound=0, cat='Integer')

# Create LP model
model = LpProblem("Shipping_Cost_Minimization", LpMinimize)

# Objective: Minimize total shipping cost
model += lpSum(
    df[(df['SKU'] == sku) & (df['Supplier name'] == supplier)]['Shipping costs'].values[0] * x[(sku, supplier)]
    for (sku, supplier) in routes if not df[(df['SKU'] == sku) & (df['Supplier name'] == supplier)].empty
)

# Add constraints (example: max 100 units per SKU)
for sku in selected_skus:
    model += lpSum(x[(sku, supplier)] for supplier in df['Supplier name'].unique()) <= 100, f"MaxUnits_{sku}"

# Solve
model.solve()

# Results
print(f"\nStatus: {LpStatus[model.status]}")
for (sku, supplier) in routes:
    if x[(sku, supplier)].value() > 0:
        print(f"Ship SKU {sku} from {supplier}: {x[(sku, supplier)].value()} units")
print(f"\nTotal Shipping Cost: ₹{model.objective.value()}")

Dataset Preview:
  Product type   SKU  Shipping costs Supplier name Location
0     haircare  SKU0        2.956572    Supplier 3   Mumbai
1     skincare  SKU1        9.716575    Supplier 3   Mumbai
2     haircare  SKU2        8.054479    Supplier 1   Mumbai
3     skincare  SKU3        1.729569    Supplier 5  Kolkata
4     skincare  SKU4        3.890548    Supplier 1    Delhi

Status: Optimal

Total Shipping Cost: ₹0.0
